In [1]:
import pandas as pd
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, Dense, Activation, Flatten, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import SGD

In [2]:
import spacy
from nltk.tokenize import word_tokenize
# loading english language model of spaCy
en_model = spacy.load('en_core_web_sm')
# gettign the list of default stop words in spaCy english model
stopwords = en_model.Defaults.stop_words

In [3]:
#Add custom stopwords
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [4]:
df = pd.read_csv(r'C:\Users\joshu\Downloads\WikiQACorpus\WikiQACorpus\WikiQA.tsv', sep='\t')

In [5]:
df.columns

Index(['QuestionID', 'Question', 'DocumentID', 'DocumentTitle', 'SentenceID',
       'Sentence', 'Label'],
      dtype='object')

In [6]:
df = pd.read_csv(r"C:\Users\joshu\Downloads\twcs.csv\twcs.csv")
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


# Thought 2
Lets look at AppleSupport AmazonHelp, SpotifyCares TMobileHelp comcastcares

# Thought 1
Chatbot analyzes whether the text input is a question ?

If question, lookup answer

If not question, get sassy (humor)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 7 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tweet_id                 int64  
 1   author_id                object 
 2   inbound                  bool   
 3   created_at               object 
 4   text                     object 
 5   response_tweet_id        object 
 6   in_response_to_tweet_id  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 131.4+ MB


In [8]:
a_df = df[df['author_id'].isin(['AppleSupport', 'AmazonHelp', 'SpotifyCares','TMobileHelp', 'comcastcares'])]

In [9]:
#This is the questions being asked to the companys (our chatbot)
#TODO add the rest of the Company Names to the list comprehension
comp_df = df[df['in_response_to_tweet_id'].isna() & (df['text'].str.contains('@sprintcare') | df['text'].str.contains('@AppleSupport') | df['text'].str.contains('@AmazonHelp')) ]

In [ ]:
comp_df.to_csv('comp_df.csv')

# Intent Categories
1. Marketing
2. Commerce
3. Software
4. Tableau
5. Employees
6. Contact Management

# Thoughts 3
k means clustering (7) to the data.
Assign an intent based on the results

In [79]:
#id / tweet
df_id_tweet = comp_df[['tweet_id', 'text']]

Thoughts 4

TODO need to select a Color scheme for the graphs

In [80]:
df_id_tweet['tokens'] = df_id_tweet['text'].apply(lambda x: word_tokenize(x))

C:\Users\joshu\AppData\Local\Temp/ipykernel_39744/1944919945.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id_tweet['tokens'] = df_id_tweet['text'].apply(lambda x: word_tokenize(x))


In [81]:
#Add custom stopwords 'AppleSupport', 'AmazonHelp', 'SpotifyCares','TMobileHelp', 'comcastcares'
swords = ['AppleSupport', 'AmazonHelp','SpotifyCares','comcastcares','TMobileHelp','sprintcare']
for word in swords:
    stopwords.add(word)


In [82]:
def removeStopWords(token):
    '''Return only tokens not in stopwords'''
    results = [word for word in token if not word in stopwords]
    results = [word for word in results if not word in string.punctuation]    
    return results

In [83]:
df_id_tweet['cleanedTokens'] = df_id_tweet['tokens'].apply(removeStopWords)

C:\Users\joshu\AppData\Local\Temp/ipykernel_39744/2869727416.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id_tweet['cleanedTokens'] = df_id_tweet['tokens'].apply(removeStopWords)


In [84]:
freq_count = dict()
for tweet in df_id_tweet['cleanedTokens']:
    for word in tweet:
        if len(word) < 2:
            continue
        elif word not in freq_count:
            freq_count[word] = 1
        elif word in freq_count:
            freq_count[word] +=1


Get top 20 Frequency Counts of words

In [85]:
import matplotlib.pyplot as plt
lists = pd.DataFrame(sorted(freq_count.items()), columns=['name','count'])

In [94]:
lists[lists['name'].isin(['employee','sales','marketing'])]

,name,count
49377,employee,39
57632,marketing,14
65393,sales,45


In [69]:
lists.sort_values(by='count', ascending=False).head(20)

,name,count
53615,https,16652
61343,phone,10993
53932,iPhone,9640
18374,115858,7878
31269,I️,7733
70596,update,7417
53809,iOS,6738
53134,help,5626
50940,fix,5434
59148,new,4844


In [63]:
#Marketing Labels
comp_df[comp_df['text'].str.contains('marketing')]

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
91481,116012,141837,True,Thu Nov 23 21:19:53 +0000 2017,@AppleSupport In how many years will I have th...,116011,NaN
244594,282445,183289,True,Fri Oct 06 12:13:19 +0000 2017,Some people are doing black marketing by selli...,"282444,282446",NaN
276989,317459,191766,True,Sat Oct 07 08:44:48 +0000 2017,@AmazonHelp I had created an account on Amazon...,317457,NaN
279518,320084,183289,True,Sat Oct 07 11:08:43 +0000 2017,How Amazon staff involved in Black marketing b...,"320083,320085",NaN
320559,367451,203041,True,Sun Oct 08 20:35:33 +0000 2017,"""Your card is ready to dispatch"" Don't fool, g...",367449,NaN
369643,422501,215625,True,Tue Oct 10 09:24:12 +0000 2017,@AmazonHelp @115850 : Stop using such marketin...,422499,NaN
578488,650686,274738,True,Wed Nov 22 21:20:37 +0000 2017,"Hey @sprintcare, I’m part of your network. Yes...",650685,NaN
640259,717057,291665,True,Tue Oct 10 18:25:50 +0000 2017,@115858 @AppleSupport: after download of IOS 1...,717055,NaN
705976,789731,183541,True,Thu Oct 12 11:29:37 +0000 2017,@AmazonHelp @115850 @115851 \nPhrase: Big RIC...,"789729,789733",NaN
855338,951953,345835,True,Fri Oct 13 21:44:06 +0000 2017,If u can't exchange products in Visakhapatnam ...,"951951,951956",NaN


In [11]:
#Cleanup data
df['in_response_to_tweet_id'].dropna().astype(int)

0                3
1                1
2                4
3                5
4                6
            ...   
2811766     139627
2811767    2987946
2811769    2987948
2811771    2812239
2811772    2987950
Name: in_response_to_tweet_id, Length: 2017439, dtype: int32

In [10]:
a#Import SpaCy language
from spacy.lang.en import English
nlp = English()

In [11]:
#Tokenize Sentences



In [26]:
authors = list(df['author_id'].unique())

In [28]:
#replies
replies = df[df['text'].str.contains('@sprintcare')]

In [31]:
replies.text = replies.text.astype(str)


C:\Users\joshu\anaconda3\envs\chatbot\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [22]:
a_df = df[df['author_id'].isin(['AppleSupport', 'AmazonHelp','Uber_suport','sprintcare', 'VerizonSupport'])]